# 1. Entorno

In [1]:
#Creamos un Notebook en Python sobre JupyterLab

# 2. Librerías

In [2]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# 3. Inicio de Sesión sobre SageMaker

In [3]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

In [4]:
#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

In [5]:
#Verificamos
print(region)

us-east-1


In [6]:
#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

# 4. Lectura de archivos

In [7]:
#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

In [8]:
#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdatestXXX"

In [9]:
#Lectura de datos de entrenamiento
dataTrain = TrainingInput(
    f"s3://{bucket}/data/insurance_train/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

In [10]:
#Lectura de datos de validación
dataTest = TrainingInput(
    f"s3://{bucket}/data/insurance_train/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

# 5. Configuración del Algoritmo

In [11]:
#Definimos el nombre para nuestro job de entrenamiento
nombreDeJobDeEntrenamiento = "entrenamiento-prediccion-numerica"

In [12]:
#Definimos el algoritmo que queremos usar
algoritmo = "linear-learner"

In [13]:
#Definimos el tipo de predicción que hará el algoritmo
#Para una predicción numérica debemos definir "regressor"
tipoDePrediccion = "regressor"

In [14]:
#Definimos la cantidad de servidores
#Con 1, nuestro algoritmo se ejecutará en 1 servidor
#Con más de 1, nuestro algoritmo se ejecutará en un clúster de varios servidores
numeroDeServidores = 1

In [15]:
#Definimos el tipo de servidor
tipoDeServidor = "ml.m5.large"

In [16]:
#Cantidad de features
cantidadDeFeatures = 11

In [17]:
#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

In [18]:
#Definimos el entrenador del algoritmo
entrenador = Estimator(
    image_uri = sagemaker.image_uris.retrieve(algoritmo, region), #Descargamos la implementación del algoritmo desde la región donde trabajamos
    role = rol, #Rol que ejecuta servicios sobre AWS
    instance_count = numeroDeServidores, #Cantidad de servidores de entrenamiento
    instance_type = tipoDeServidor, #Tipo de servidor de entrenamiento
    predictor_type = tipoDePrediccion, #Tipo de predicción del algoritmo
    sagemaker_session = sesion, #Sesión de SageMaker
    base_job_name = nombreDeJobDeEntrenamiento #Nombre del job de entrenamiento
)

In [19]:
#Configuramos los parametros del algoritmo
entrenador.set_hyperparameters(
    feature_dim = cantidadDeFeatures, #Cantidad de features
    predictor_type = tipoDePrediccion, #Indicamos que tipo de predicción es
    normalize_data = "true", #Normalizamos los features
    normalize_label = "true" #Normalizamos el label
)

# 6. Entrenamiento del Modelo

In [20]:
#Entrenamos y validamos el modelo
#MIENTRAS SE ENTRENA EL MODELO: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver cómo el modelo se entrena
#TIEMPO DE ENTRENAMIENTO: 5 MINUTOS
entrenador.fit({"train": dataTrain, "validation": dataTest})

INFO:sagemaker:Creating training-job with name: entrenamiento-prediccion-numerica-2024-07-04-15-26-44-591


2024-07-04 15:26:45 Starting - Starting the training job...
2024-07-04 15:26:59 Starting - Preparing the instances for training...
2024-07-04 15:27:24 Downloading - Downloading input data...
2024-07-04 15:28:09 Downloading - Downloading the training image........Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/04/2024 15:29:25 INFO 140615955126080] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta':

# 7. Estadísticas del Modelo

In [21]:
#Librería para manipular los servicios de AWS
import boto3

In [22]:
#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

In [23]:
#Obtenemos el nombre del último entrenamiento realizado
nombreDeEntrenamiento = entrenador.latest_training_job.name

#Verificamos
print(nombreDeEntrenamiento)

entrenamiento-prediccion-numerica-2024-07-04-15-26-44-591


In [24]:
#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDeEntrenamiento)

In [25]:
#Vemos toda la descripción de entrenamiento
descripcionDeEntrenamiento

{'TrainingJobName': 'entrenamiento-prediccion-numerica-2024-07-04-15-26-44-591',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:380460860730:training-job/entrenamiento-prediccion-numerica-2024-07-04-15-26-44-591',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-380460860730/entrenamiento-prediccion-numerica-2024-07-04-15-26-44-591/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'feature_dim': '11',
  'normalize_data': 'true',
  'normalize_label': 'true',
  'predictor_type': 'regressor'},
 'AlgorithmSpecification': {'TrainingImage': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:progress',
    'Regex': '#progress_metric: host=\\S+, completed (\\S+) %'},
   {'Name': 'validation:mae',
    'Regex': '#quality_metric: host=\\S+, validation mae <loss>=(\\S+)'},
   {'Name': 'train:objective_loss',
    'Regex': '#quality_metr

In [26]:
#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
#El error cuadrático medio se encuentra en "validation:mse"
#El r2 (error "porcentual") se encuentra en "validation:r2"
#IMPORTANTE: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver las estadísticas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

[{'MetricName': 'train:progress',
  'Value': 100.0,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'validation:mae',
  'Value': 3968.21044921875,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'train:objective_loss',
  'Value': 0.5741294622421265,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss',
  'Value': 107166480.0,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss:final',
  'Value': 40647928.0,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'validation:rmse',
  'Value': 6375.57275390625,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricName': 'validation:mse',
  'Value': 40647928.0,
  'Timestamp': datetime.datetime(2024, 7, 4, 15, 29, 30, tzinfo=tzlocal())},
 {'MetricNa